#  Tutorial: Patchworking real tropical hypersurfaces

Viro's combinatorial patchworking method
takes as its input a regular subdivision $\tau$ of a dilated standard simplex $d\cdot\Delta_n$ (where $\Delta_n = \mathrm{conv}\{e_1,\ldots,e_n\}$), and a sign distribution on its vertices $s:\tau_0\rightarrow\mathbb Z/2\mathbb Z$,
and builds a polyhedral hypersurface (a pure codimension 1 polyhedral complex), that is of the same topological type as some real algebraic hypersurface of degree $d$ (i.e., a real algebraic variety $\mathbb RV(F)$ for some real polynomial $F$ with $\mathrm{deg}F=d$).

We present an implementation of this method using real tropical hypersurfaces,
with the induced regular subdivision of its Newton polytope taking the place of the above mentioned triangulation,
as well as an efficient algorithm to compute the $\mathbb Z/2\mathbb Z$ homology of the resulting hypersurface, which also works for non-regular subdivisions (see below).

For the math, see for example [Viro](https://arxiv.org/abs/math/0611382) or, for the case of tropical hypersurfaces, [Mikhalkin](https://arxiv.org/abs/math/0403015).  This polymake implementation is discussed in
* Joswig & Vater: Real tropical hyperfaces by patchworking in polymake, in: Mathematical software – ICMS 2020 (Bigatti et al, eds.), Springer, LNCS 12097, [doi:10.1007/978-3-030-52200-1_20](htpps://doi.org/10.1007/978-3-030-52200-1_20).

## Nomenclature

In the case of a real tropical hypersurface $T_s(f)$ (i.e., a tropical hypersurface $T(f)$ together with a sign distribution $s$ on its support), patchworking works by symmetrically gluing together $2^n$ copies of $H$ (one for each orthant of the ambient space $\mathbb R^n$), and then forgetting certain facets of these copies (this depends on $s$).

The quantity encoding which facets are remembered in which orthant is called its *real phase structure*.
We call the resulting polyhedral hypersurface the corresponding *patchworked hypersurfaces*, or simply a *realization* of $T_s(f)$.

## Basic Functionality

Naturally, we'll want to load Polymake's `tropical` application first:

In [6]:
application "tropical";

The simplest non-trivial example builds a polyhedral line from a tropical line.
So let's first create a tropical line:

In [7]:
$h1 = new Hypersurface<Min>(POLYNOMIAL=>toTropicalPolynomial("min(x,y,z)"));
print $h1->MONOMIALS;

(3) (2 1)
(3) (1 1)
(3) (0 1)


We can now add a sign distribution on its monomials in the following way:

In [8]:
$h1_pw1 = $h1->PATCHWORK(SIGNS=>[0,0,1]); # a real structure on $h1
$h1_pw2 = $h1->PATCHWORK(SIGNS=>[1,0,1]); # another one, why not?

The property `PATCHWORK` represents a multiple subobject to a `Hypersurface` object (whether we deal with a `Min` or `Max` tropical hypersurface is irrelevant for the patchworking), parametrized by the property `SIGNS` with type `Array<Bool>`, which represents the sign distribution on its support.
This subobject carries the information about the real tropical hypersurface.

Note that this allows us to create multiple "real structures" on the same tropical hypersurface object (as we did in the above example).

In [9]:
print $h1->MAXIMAL_POLYTOPES; # the facets of the (original) tropical hypersurface

{0 1}
{0 2}
{0 3}


In [10]:
print $h1_pw1->REAL_PHASE;

{1 2}
{0 1}
{0 2}


The property `REAL_PHASE` is an `IncidenceMatrix`, representing the real phase structure of the real tropical hypersurface, with the following semantics:

Each orthant $o\subset\mathbb R^n$ corresponds to a subset $o\cap\{-e_1,\ldots,-e_n\}\subset\{-e_1,\ldots,-e_n\}$, whose characteristic vector is the binary representation of a unique natural number $0\leq N_o< 2^n$.
E.g. in the plane, the positive orthant has binary representation $(00)_2 = 0$, and the negative $(11)_2 = 3$.

The $j$-th entry in the $i$-th row of `REAL_PHASE` is now $1$ iff the $i$-th facet of the tropical hypersurface (represented by the $i$-th row of `MAXIMAL_POLYTOPES`; see above) is remembered in the orthant with binary representation $j$.

### Realization as a `PolyhedralComplex`

We can produce a realization of the patchworked hypersurface as a `PolyhedralComplex` with the user method `...->realize()`:

In [11]:
$h1_pw_r = $h1_pw1->realize();
print $h1_pw_r->COMBINATORIAL_DIM;
svg($h1_pw_r->VISUAL);

1

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.0//EN" "http://www.w3.org/TR/2001/REC-SVG-20010904/DTD/svg10.dtd">
 
 pcom:h1_pw_r 
 
 
 
 
 
 
 
 
 
 
	<!-- 
	Generated using the Perl SVG Module V2.64
	by Ronan Oger
	Info: http://www.roitsystems.com/
 -->

Click here for additional output 
 
polymake: used package SVG
 Generated using the Perl SVG Module
 by Ronan Oger

Note that the realization is only unique up to combinatorial equivalence.

### Some classic curves

There are convenience functions to produce some classic algebraic curves:

In [12]:
$harnack = harnack_curve(6); # harnack curve of degree 6
$ragsdale = ragsdale_counterexample(); # a counterexample to the ragsdale conjecture
$gudkov = gudkov_curve(); # gudkov curve (degree 6)

These return a `Hypersurface` object with a unique `PATCHWORK` property attached, to access it simply use `...->PATCHWORK`.

In [13]:
print $ragsdale->PATCHWORK->SIGNS;
svg($ragsdale->PATCHWORK->realize("uniform")->VISUAL);

1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 1 0 1 0 0 0 0 1 0 1 0 0 1

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.0//EN" "http://www.w3.org/TR/2001/REC-SVG-20010904/DTD/svg10.dtd">
 
 pcom: 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
	<!-- 
	Generated using the Perl SVG Module V2.64
	by Ronan Oger
	Info: http://www.roitsystems.com/
 -->

## The $\mathbb Z/2\mathbb Z$ cellular homology of a patchworked hypersurface

Naive computation of the homology of a patchworked hypersurface is computationally expensive, since its size depends exponentially on the dimension (we glue together $2^n$ copies of the tropical hypersurface).

Shaw & Renaudineau describe a way to exploit the symmetry of a patchworked hypersurface to compute its $\mathbb Z/2\mathbb Z$ homology directly from the triangulation + sign distribution, without needing to construct a polyhedral complex realization; cf. [arXiv:1805.02030](https://arxiv.org/abs/1805.02030).

This leads to a straight forward algorithm, which is implemented in Polymake, and accessible in the following way:

In [14]:
print $h1_pw1->CHAIN_COMPLEX_Z2;

<1 0 1
0 1 1
1 1 0
>


In [15]:
print $h1_pw1->BETTI_NUMBERS_Z2;

1 1

## Non-regular patchworking

It is possible to use this implementation starting with a (possibly non-regular) subdivision $\tau$ (i.e., without the data of a tropical hypersurface).

The usage is similar, you just have to supply the `DUAL_SUBDIVISION` property of the hypersurface (even if mathematically no such hypersurface exists, i.e. in the non-regular case), instead of the data of the tropical `POLYNOMIAL`.

Here is such a non-regular subdivision of $4\Delta_2$, the "mother of all examples":

In [17]:
$moae = new fan::SubdivisionOfPoints(POINTS=>[[1,4,0,0],[1,0,4,0],[1,0,0,4],[1,2,1,1],[1,1,2,1],[1,1,1,2]],
                                     MAXIMAL_CELLS=>[[0,1,3],[0,2,5],[0,3,5],[1,2,4],[1,3,4],[2,4,5],[3,4,5]]);
$h2 = new Hypersurface<Min>(DUAL_SUBDIVISION=>$moae);
$h2_pw = $h2->PATCHWORK(SIGNS=>[0,0,0,1,1,0]);
print $h2_pw->BETTI_NUMBERS_Z2;

3 3